In [1]:
import llvmlite
import numba

llvm_type = llvmlite.ir.IntType(64)


class TestStructType(numba.types.Type):
    def __init__(self):
        super().__init__(name="TestStruct")


test_struct_type = TestStructType()


def as_ptr(builder, value):
    new = builder.alloca(llvm_type)
    builder.store(value, new)
    return new
        

@numba.extending.register_model(test_struct_type)
class TestStructModel(numba.extending.models.PrimitiveModel):
    def __init__(self, dmm, fe_type):
        super().__init__(dmm, fe_type, llvmlite.ir.PointerType(llvm_type))

    def get_return_type(self):
        return llvm_type

#     def get_data_type(self):
#         return ptr_llvm_type

#     def as_argument(self, builder, value):
#         return as_ptr(builder, value)

    def as_return(self, builder, value):
        return builder.load(value)

#     def as_data(self, builder, value):
#         return as_ptr(builder, value)

#     def from_argument(self, builder, value):
#         return builder.load(value)

    def from_return(self, builder, value):
        return as_ptr(builder, value)


@numba.extending.intrinsic
def create_test_struct(typingctx, int_t):

    sig = test_struct_type(int_t)

    def codegen(context, builder, sig, args):
        a = builder.alloca(llvm_type)
        builder.store(value=args[0], ptr=a)
        return a

    return sig, codegen


@numba.extending.intrinsic
def convert_test_struct(typingctx, test_struct_t):

    sig = numba.types.int64(test_struct_t)

    def codegen(context, builder, sig, args):
        return builder.load(args[0])

    return sig, codegen


@numba.njit
def inner_fn_works():
    a = create_test_struct(10)
    print("inner_fn_works", convert_test_struct(a))
    return a


@numba.njit
def outer_fn_works():
    print("outer_fn_works", convert_test_struct(inner_fn_works()))


@numba.njit
def inner_fn_broken():
    a = create_test_struct(10)
    return a


@numba.njit
def outer_fn_broken():
    print("outer_fn_broken", convert_test_struct(inner_fn_broken()))


outer_fn_works()
outer_fn_broken()

AssertionError: 